In [1]:
from osgeo import gdal,ogr,osr
import os
import numpy as np
import matplotlib.pyplot as plt
#import spectral
#import rasterio
#import spectral.io.envi as envi
#import json
import math
from math import sqrt
import pandas as pd
import geopandas as gpd
import affine
from affine import Affine
import numpy as np
from osgeo import gdal,ogr,osr
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [2]:
file = "data/ang20180729t212542_rfl_v2r2_img"
hdr_file = f"{file}.hdr"
output_dir = os.path.join(os.getcwd(),"output")

In [3]:
# Step 1: Open the hyperspectral image
img_file = file # Replace with your file path
img_open = gdal.Open(img_file)

# Step 2: Read the reflectance data (425 bands, 660 cols, 3647 rows)
bands = img_open.RasterCount
cols = img_open.RasterXSize
rows = img_open.RasterYSize

# Reading the reflectance data for each band (3D matrix of shape [bands, cols, rows])
#reflectance_data = np.zeros((bands, rows, cols), dtype=np.float32)  # Fixed shape to [bands, rows, cols]

reflectance_data = img_open.ReadAsArray()
reflectance_data = np.where(reflectance_data == -9999, np.nan, reflectance_data)
#for band in range(bands):
#    reflectance_data[band] = img_open.GetRasterBand(band + 1).ReadAsArray()

In [4]:
reflectance_data.shape

(425, 3647, 660)

In [5]:
# Step 3: Reshape the data for similarity computation
# Each pixel has a 1D vector of size 425 (number of bands), so we need to reshape
# it into shape (num_pixels, num_bands), where each row corresponds to a pixel's reflectance
pixels = reflectance_data.reshape((bands, -1)).T  # Shape: [num_pixels, 425]

In [6]:
# Step 4: Check for NaN values before attempting to remove
print(f"NaNs in the original data: {np.sum(np.isnan(pixels))}")

# Step 5: Remove rows (pixels) with any NaN values
pixels_clean = pixels[~np.isnan(pixels).any(axis=1)]  # Remove rows with any NaN values

# Step 6: Ensure no NaN values are left
print(f"NaNs after removal: {np.sum(np.isnan(pixels_clean))}")

NaNs in the original data: 79099725
NaNs after removal: 0


In [7]:
pixels_clean.shape

(2220903, 425)

In [8]:
# Step 4: Normalize the data (Optional but recommended for cosine similarity)
pixels_normalized = normalize(pixels_clean)

In [10]:
print(f"NaNs of pixels_normalized: {np.sum(np.isnan(pixels_normalized))}")

NaNs of pixels_normalized: 0


In [11]:
# Step 5: Compute the Cosine Similarity Matrix between all pixels
similarity_matrix = cosine_similarity(pixels_normalized)

: 

In [ ]:
# Step 6: Clustering similar pixels based on cosine similarity
num_clusters = 5  # Define the number of clusters (similarity groups)
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
labels = kmeans.fit_predict(similarity_matrix)

In [ ]:
# Step 7: Reshape labels to match the image's spatial dimensions (rows, cols)
labels_image = labels.reshape((rows, cols))  # Now labels should match the original image dimensions

In [ ]:
# Step 8: Visualize the resulting clusters (grading of pixels)
plt.figure(figsize=(10, 6))
plt.imshow(labels_image, cmap='tab10')  # You can adjust the colormap
plt.title("Pixel Similarity Grading (Cosine Similarity)")
plt.colorbar(label="Cluster ID")
plt.show()

In [ ]:
# Step 9: Optionally, save the result (graded pixel clusters) to a new file
# You can save the graded result back to a raster file (e.g., GeoTIFF)
driver = gdal.GetDriverByName('GTiff')
output_file = "graded_image.tif"
out_dataset = driver.Create(output_file, cols, rows, 1, gdal.GDT_Int32)
out_dataset.GetRasterBand(1).WriteArray(labels_image)
out_dataset.SetGeoTransform(img_open.GetGeoTransform())
out_dataset.SetProjection(img_open.GetProjection())
out_dataset.FlushCache()

In [10]:
# open the hyperspectral image file with GDAL
img_open = gdal.Open(file)

In [18]:
uesche = img_open.ReadAsArray()

In [ ]:
uesche.shape

In [20]:
uesche = np.where(uesche == -9999, np.nan, uesche)

In [24]:
pixel_reflectance = uesche[:, 800, 600]

In [ ]:
# read a few key properties of the image
nbands = img_open.RasterCount
ncols = img_open.RasterXSize
nrows = img_open.RasterYSize

print("\n".join(["Bands:\t"+str(nbands),"Cols (x-axis):\t"+str(ncols),"Rows (y-axis):\t"+str(nrows)]))

In [ ]:
# read and display band 57 just to have a quick look at the image
img_red = img_open.GetRasterBand(1).ReadAsArray()
# Set no-data values (-9999) to NaN
img_red = np.where(img_red == -9999, np.nan, img_red)
plt.rcParams["figure.figsize"] = [2,12]
plt.rcParams["figure.dpi"] = 100
plt.imshow(img_red,vmin=0, vmax=0.1)
plt.colorbar()
plt.show()

In [ ]:
# Read header file and get the metadata
def read_envi_header(file_path):
    metadata = {}
    with open(file_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            if " = " in line:
                key, value = line.strip().split(" = ", 1)
                metadata[key.strip()] = value.strip().strip('{}')
            elif line.strip().endswith('}'):
                # Multi-line metadata
                key, value = lines[lines.index(line) - 1].strip().split(" = ", 1)
                metadata[key.strip()] += line.strip().strip('{}')
    return metadata

metadata = read_envi_header(hdr_file)
metadata

In [ ]:
# save a few key meta data features into variables
map_info = metadata.get("map info")
print(f"Map info: {map_info}")

samples = metadata.get("samples")
print(f"Samples: {samples}")

wavelength = metadata.get("wavelength")
print(f"Wavelength: {wavelength}")

In [ ]:
# define plot site file path and read as a geopandas df
plot_site_input_dir = os.path.join(os.getcwd(),"input","32_Prudhoe_Bay_ARCSEES_Road_Study_0")
plot_site_file = os.path.join(plot_site_input_dir,"32_Prudhoe_Bay_ARCSEES_Road_Study_0.shp")
print(plot_site_file)

# open with ogr
driver = ogr.GetDriverByName("ESRI Shapefile")
plot_sites = driver.Open(plot_site_file, 0)

plot_sites_geojson = gpd.read_file(plot_site_file)
plot_sites_geojson.head(5)

In [ ]:


# Iterate using itertuples
for row in plot_sites_geojson.itertuples():
    print(f"Index: {row.Index}")
    print(f"Field_rele: {row.Field_rele}")
    print(f"Modified_r: {row.Modified_r}")
    print(f"Plant_comm: {row.Plant_comm}")
    print(f"Habitat_Ty: {row.Habitat_Ty}")
    print(f"Longitude: {row.lon}")
    print(f"Latitude: {row.lat}")
    print(f"geometry: {row.geometry}")
    print('---')


In [ ]:
# shapefiles have a nested structure: layer(s) -> feature(s) -> geometry
lyr = plot_sites.GetLayer() # get the only layer in the shapefile
feat = lyr.GetFeature(1)          # get the first feature in the layer (1 feature per site)
geom = feat.GetGeometryRef()      # get the feature's geometry
geo_transform = img_open.GetGeoTransform() # get hyperspectral image geotransformation

# get transform for decimal degrees
from_srs = lyr.GetSpatialRef()                                         # get shapefile srs def
to_srs = osr.SpatialReference()                                        # init ogr srs object

to_srs.ImportFromEPSG(4326)                                            # import wgs84 srs def
xytransform = osr.CoordinateTransformation(from_srs,to_srs)            # get transform object

# get UTM and lat/long coordinates for each of the sites
utm_coordinate_pairs = {}
ll_coordinate_pairs = {}
for feature in lyr:
    geometry = feature.GetGeometryRef()                                 # get site geometry
    utm_coordinate_pairs[feature['Modified_r']] = (geometry.GetX(), geometry.GetY()) # get x,y utm coordinates 
    geometry.Transform(xytransform)                                      # to wgs84
    ll_coordinate_pairs[feature['Modified_r']] = (geometry.GetX(), geometry.GetY())  # get lon, lat

   
# get the x and y UTM coordinates for the first site
# utm coordinates twisted in dataset
x, y = utm_coordinate_pairs['T1-010-C']

x_res = sqrt(geo_transform[1]*geo_transform[1] + geo_transform[2]*geo_transform[2])
y_res = sqrt(geo_transform[4]*geo_transform[4]+ geo_transform[5]*geo_transform[5])

print(f"x_res: {x_res}")
print(f"y_res: {y_res}")
geo_transform_new = (geo_transform[0],x_res,geo_transform[2],geo_transform[3],geo_transform[4],y_res)


# Create affine transformation matrix
affine_transform = Affine(
    geo_transform_new[1], geo_transform_new[2], geo_transform_new[0],
    geo_transform_new[4], geo_transform_new[5], geo_transform_new[3]
)


affine_transform = affine.Affine.from_gdal(*img_open.GetGeoTransform())     # affine forward transform
inverse_transform = ~affine_transform                                  # invert transform
px, py = inverse_transform * (x, y)                                    # apply to x,y coordinates
px, py = int(px + 0.5), int(py + 0.5)                                  # get new x,y as integers


# print the three coordinates (UTM, geographic, image)
print( "\n".join(["Site 1 UTM coordinates (x,y): "+"\t"*4+str((x,y)),
       " are equal to geographic coordinates (lng,lat): \t"+str(ll_coordinate_pairs['T1-005-C']),
       " and fall within image coordinates (pixel,line):\t"+str((px,py))]) )

In [ ]:
geo_transform

In [13]:
# Dictionary containing band descriptions
band_dictionary = {
    "visible-violet": {'lower': 375, 'upper': 450, 'color': 'violet'},
    "visible-blue": {'lower': 450, 'upper': 485, 'color': 'blue'},
    "visible-cyan": {'lower': 485, 'upper': 500, 'color': 'cyan'},
    "visible-green": {'lower': 500, 'upper': 565, 'color': 'green'},
    "visible-yellow": {'lower': 565, 'upper': 590, 'color': 'yellow'},
    "visible-orange": {'lower': 590, 'upper': 625, 'color': 'orange'},
    "visible-red": {'lower': 625, 'upper': 740, 'color': 'red'},
    "near-infrared": {'lower': 740, 'upper': 1100, 'color': 'gray'},
    "shortwave-infrared": {'lower': 1100, 'upper': 2500, 'color': 'white'}
}

# Function to classify bands based on their wavelength
def classify_band(wavelength):
    for region, limits in band_dictionary.items():
        if limits['lower'] < wavelength <= limits['upper']:
            return region
    return None

# define site on image
sitename1 = "Testsite 1"
px1, py1 = 600, 6000

sitename2 = "Testsite 2"
px2, py2 = 500, 5000

# Extracting metadata keys and values
metadata_keys = img_open.GetMetadata().keys()
metadata_values = img_open.GetMetadata().values()


# Lists of band numbers and band centers
band_numbers = [
    int(key.split("_")[1])
    for key in metadata_keys
    if key != "wavelength_units"
]

band_centers = [
    float(value.split(" ")[0])
    for value in metadata_values
    if value != "Nanometers"
]

# Classify each band center
em_regions = [classify_band(b) for b in band_centers]

def get_pixel_reflectance(x,y):
    return img_open.ReadAsArray(x,y,1,1).flatten().tolist()


# get the reflectance values of the site (careful: these values are already sorted --> add to df after sort)
#testsite1_reflectance = img_open.ReadAsArray(px1,py1,1,1).flatten().tolist()
#testsite2_reflectance = img_open.ReadAsArray(px2,py2,1,1).flatten().tolist()
testsite1_reflectance = get_pixel_reflectance(px1,py1)
testsite2_reflectance = get_pixel_reflectance(px2,py2)

reflectance_slow = [
    float(img_open.GetRasterBand(band_nr).ReadAsArray()[py,px])
    for band_nr in band_numbers
]

In [ ]:
# DataFrame describing bands
bands = pd.DataFrame({
    "Band number": band_numbers,
    "Band center (nm)": band_centers,
    "EM region": em_regions
}, index=band_numbers).sort_index()

bands[f"{sitename1} Reflectance"] = testsite1_reflectance
bands[f"{sitename2} Reflectance"] = testsite2_reflectance
print(bands.head(10))


In [ ]:
bands.tail

In [ ]:
bands.query("`Band number` == 57")

### Create a reflectance plot for the 2 test sites

In [ ]:
fig, ax = plt.subplots()
ax.plot(bands["Band center (nm)"], bands["Testsite 1 Reflectance"], linewidth=1)
ax.plot(bands["Band center (nm)"], bands["Testsite 2 Reflectance"], linewidth=1)

x_limit_min = math.floor(bands["Band center (nm)"].min())
x_limit_max = math.ceil(bands["Band center (nm)"].max())

y_limit_min = bands[["Testsite 1 Reflectance","Testsite 2 Reflectance"]].min().min()
y_limit_max = bands[["Testsite 1 Reflectance","Testsite 2 Reflectance"]].max().max()

y_limit_min = -0.025
y_limit_max = 0.08

ax.set(xlim=(x_limit_min,x_limit_max), ylim=(y_limit_min,y_limit_max), xlabel="wavelength", ylabel="reflectance")

### Create the same plot with the indicators of the wavelength

In [ ]:
%matplotlib inline
titlefont = {'fontsize':16,'fontweight':2,
             'verticalalignment':'baseline','horizontalalignment':'center'}
plt.rcParams['figure.figsize'] = [16, 8]

# plot. all of this is matplotlib ---------->
ax1 = plt.subplot(211)
ax2 = plt.subplot(212, sharex=ax1)

bands.plot(x='Band center (nm)', y=f"{sitename1} Reflectance", 
            ax=ax1, c='black', label='T1-005-T', legend=True)
bands.plot(x='Band center (nm)', y=f"{sitename2} Reflectance", 
            ax=ax2, c='black', label='T1-005-C', legend=True)

for i, ax in enumerate([ax1,ax2]): 
    
    # add shaders for band regions
    for region,limits in band_dictionary.items():
        ax.axvspan(limits['lower'], limits['upper'], alpha=0.2, 
                   color=limits['color'], label=region)
        
    # add water vapor region shaders
    ax.axvspan(1340, 1445, alpha=0.1, color='blue', label='water vapor regions')
    ax.axvspan(1790, 1955, alpha=0.1, color='blue')
    
    # some plotting config
    ax.set_ylim(0,0.15)
    ax.set_xlim(min(band_centers),max(band_centers))
    ax.set_ylabel("reflectance", fontsize=16)
    ax.set_xlabel("wavelength (nm)", fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.grid('on', alpha=0.25)
    ax.set_title("Test site "+str(i+1), fontdict = titlefont, pad = 10)

# now add custom legend
ax1.legend(prop={'size': 12}, loc = 'upper right', 
           bbox_to_anchor=(1, 1.25), ncol = 2, framealpha = 1)





In [29]:
# remove a few band to reduce atmospheric noise 
wavelengths = bands["Band center (nm)"]
bblist = np.ones((425,))
bblist[0:14] = 0
bblist[189:225] = 0
bblist[281:336] = 0
bblist[405:] = 0

In [31]:
# set two (random) test sites within the image boundaries
px1_x, px1_y = 100, 6000
px2_x, px2_y = 250, 510

# Re-plot with the bad bands list and wavelengths applied
pixel1 = img_open.ReadAsArray(px1_x, px1_y, 1, 1)
pixel1 = np.reshape(pixel1,(425))

pixel2 = img_open.ReadAsArray(px2_x, px2_y, 1, 1)
pixel2 = np.reshape(pixel2,(425))

pixel1[bblist == 0] = np.nan
pixel2[bblist == 0] = np.nan


In [ ]:
min_pixel1 = np.nanmin(pixel1)
min_pixel2 = np.nanmin(pixel2)

if(min_pixel1 > min_pixel2):
    y_min = min_pixel2
else:
    y_min = min_pixel1

max_pixel1 = np.nanmax(pixel1)
max_pixel2 = np.nanmax(pixel2)

if(max_pixel1 > max_pixel2):
    y_max = max_pixel1
else:
    y_max = max_pixel2

print(y_min)
print(y_max)

In [ ]:
# Compare vegetation and water spectra
plt.plot(wavelengths, pixel1, color = 'g',  label = 'T1-005-T')
plt.plot(wavelengths, pixel2, color = 'b',  label = 'T1-005-C')
plt.xlabel ( 'Wavelength (nm)', fontsize=20)
#plt.ylim(y_min, y_max)
plt.ylabel('Reflectance', fontsize=20)
plt.grid()
plt.legend()


plt.show()

In [ ]:
%matplotlib inline
titlefont = {'fontsize':16,'fontweight':2,
             'verticalalignment':'baseline','horizontalalignment':'center'}
plt.rcParams['figure.figsize'] = [16, 8]

plt.plot(wavelengths, pixel1, color = 'g',  label = 'T1-005-T')
plt.plot(wavelengths, pixel2, color = 'b',  label = 'T1-005-C')


# add shaders for band regions
for region,limits in band_dictionary.items():
    plt.axvspan(limits['lower'], limits['upper'], alpha=0.2, color=limits['color'], label=region)
        
# add water vapor region shaders
plt.axvspan(1320, 1500, alpha=0.1, color='blue', label='water vapor regions')
plt.axvspan(1780, 2055, alpha=0.1, color='blue')
    
# some plotting config
#plt.ylim(y_min,y_max)
#plt.xlim(min(wavelengths),max(wavelengths))
plt.ylabel("reflectance", fontsize=16)
plt.xlabel("wavelength (nm)", fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.grid('on', alpha=0.25)
plt.title("Test sites", fontdict = titlefont, pad = 10)

# now add custom legend
plt.legend(prop={'size': 12}, loc = 'upper right', 
           bbox_to_anchor=(1, 1.25), ncol = 2, framealpha = 1)